In [ ]:
import pandas as pd
import random
import copy
import matplotlib.pyplot as plt

In [ ]:
df_alimentos = pd.read_csv("alimentos.csv")

In [ ]:
METAS_NUTRICIONAIS = {
    "Energia..kcal.": 2000,
    "Proteína..g.": 75,
    "Carboidrato..g.": 275,
    "Lipídeos..g.": 70,
}

TAMANHO_POPULACAO = 30
NUM_GERACOES = 1000
NUM_ELITISTAS = 2
TAXA_MUTACAO = 0.5
TAXA_ELITISMO = 0.1
TAMANHO_TORNEIO = 3

random.seed(42)

In [ ]:
def gerar_cardapio_aleatorio(df):
    """Gera um cardápio aleatório com base nos alimentos disponíveis."""

    alimentos = df["Descrição dos alimentos"].dropna().unique().tolist()

    cardapio = {
        "cafe": random.sample(alimentos, k=3),
        "almoco": random.sample(alimentos, k=3),
        "jantar": random.sample(alimentos, k=3),
    }

    return cardapio

In [ ]:
populacao_inicial = [
    gerar_cardapio_aleatorio(df_alimentos) for _ in range(TAMANHO_POPULACAO)
]

In [ ]:
def calcular_nutrientes_cardapio(cardapio, df):
    """Calcula os nutrientes de um cardápio com base nos alimentos e nas quantidades."""

    total = {
        "Energia..kcal.": 0,
        "Proteína..g.": 0,
        "Carboidrato..g.": 0,
        "Lipídeos..g.": 0,
    }

    for refeicao, alimentos in cardapio.items():
        for alimento in alimentos:
            dados = df[df["Descrição dos alimentos"] == alimento]
            if not dados.empty:
                for chave in total.keys():
                    valor = dados[chave].values[0]
                    if pd.notna(valor):
                        total[chave] += valor

    return total

In [ ]:
def calcular_fitness(nutrientes, metas):
    """
    Calcula o fitness de um cardápio com base nos nutrientes e nas metas estabelecidas.
    Quanto menor o desvio, maior o fitness.
    """

    fitness = 0

    for nutriente in metas.keys():
        valor_real = nutrientes.get(nutriente, 0)
        valor_meta = metas[nutriente]
        desvio = abs(valor_real - valor_meta)
        fitness -= desvio

    return fitness

In [ ]:
def selecao_por_torneio(populacao, fitnesses, tamanho_torneio=TAMANHO_TORNEIO):
    """Realiza a seleção por torneio em uma população de cardápios."""

    selecionados = []

    for _ in range(len(populacao)):
        participantes = random.sample(list(zip(populacao, fitnesses)), tamanho_torneio)
        vencedor = max(participantes, key=lambda x: x[1])
        selecionados.append(vencedor[0])

    return selecionados

In [ ]:
def crossover(card1, card2):
    """Realiza o crossover (cruzamento) entre dois cardápios."""

    filho = {}

    # Para cada refeição, escolhe aleatoriamente de qual cardápio herdar (card1 e card2)
    for refeicao in ["cafe", "almoco", "jantar"]:
        lista_escolhida = random.choice([card1[refeicao], card2[refeicao]])
        filho[refeicao] = lista_escolhida.copy()

    return filho

In [ ]:
def mutacao_alimento(cardapio, df, taxa_mutacao=TAXA_MUTACAO):
    """Realiza a mutação de um alimento em um cardápio."""

    novo_cardapio = copy.deepcopy(cardapio)

    # Decide se vai ocorrer mutação (taxa_mutacao)
    if random.random() < taxa_mutacao:
        # Escolhe aleatoriamente uma refeição
        refeicoes_com_alimentos = [r for r in novo_cardapio if novo_cardapio[r]]
        if refeicoes_com_alimentos:
            refeicao_escolhida = random.choice(refeicoes_com_alimentos)
            # Escolhe aleatoriamente um alimento dessa refeição
            idx = random.randint(0, len(novo_cardapio[refeicao_escolhida]) - 1)
            # Escolhe um novo alimento possível
            alimentos_possiveis = df["Descrição dos alimentos"].dropna().unique().tolist()
            novo_cardapio[refeicao_escolhida][idx] = random.choice(alimentos_possiveis)

    return novo_cardapio

In [ ]:
def mutacao_refeicao(cardapio, df, taxa_mutacao=TAXA_MUTACAO):
    """Realiza a mutação de uma refeição em um cardápio."""

    novo_cardapio = copy.deepcopy(cardapio)

    # Decide se vai ocorrer mutação (taxa_mutacao)
    if random.random() < taxa_mutacao:
        # Escolhe aleatoriamente uma refeição
        refeicoes = list(novo_cardapio.keys())
        refeicao_escolhida = random.choice(refeicoes)
        # Escolhe aleatoriamente um alimento possível para a refeição
        alimentos_possiveis = df["Descrição dos alimentos"].dropna().unique().tolist()
        novo_cardapio[refeicao_escolhida] = random.sample(alimentos_possiveis, k=3)

    return novo_cardapio

In [ ]:
def algoritmo_genetico(
    populacao_inicial=populacao_inicial,
    df=df_alimentos,
    metas=METAS_NUTRICIONAIS,
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES,
):
    """Executa um algoritmo genético para otimizar cardápios alimentares de acordo com metas nutricionais."""

    populacao = copy.deepcopy(populacao_inicial)
    historico_fitness = []

    # Para cada geração...
    for g in range(num_geracoes):
        # Calcula o fitness de cada indivíduo da população
        fitnesses = [
            calcular_fitness(calcular_nutrientes_cardapio(c, df), metas)
            for c in populacao
        ]

        # Guarda o melhor fitness da geração atual
        historico_fitness.append(max(fitnesses))

        # Seleciona indivíduos para reprodução usando torneio
        selecionados = selecao_por_torneio(populacao, fitnesses)

        # Realiza o crossover e mutação para gerar a nova população
        nova_populacao = []
        for i in range(0, tamanho_populacao, 2):
            pai1 = selecionados[i]
            # Garante que o índice não ultrapasse o tamanho da população
            pai2 = selecionados[i + 1 if i + 1 < tamanho_populacao else 0]
            # Gera dois filhos a partir dos pais selecionados
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            # Aplica mutação nos filhos e adiciona à nova população
            nova_populacao.extend(
                [mutacao_refeicao(filho1, df), mutacao_refeicao(filho2, df)]
            )

        # Atualiza a população para a próxima geração (mantém o tamanho fixo)
        populacao = nova_populacao[:tamanho_populacao]

    # Após todas as gerações, calcula o fitness final de cada indivíduo
    fitnesses = [
        calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao
    ]

    # Encontra o índice do indivíduo com melhor fitness
    melhor_idx = fitnesses.index(max(fitnesses))

    # Retorna o melhor cardápio, seu fitness e o histórico de fitness
    return populacao[melhor_idx], fitnesses[melhor_idx], historico_fitness

In [ ]:
melhor_cardapio, melhor_fitness, historico_fitness_base = algoritmo_genetico()

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_base,
    color="red",
    linestyle="-",
    linewidth=2,
)
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Melhor Fitness por Geração")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def algoritmo_genetico_com_elitismo(
    populacao_inicial=populacao_inicial,
    df=df_alimentos,
    metas=METAS_NUTRICIONAIS,
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES,
):
    """Executa um algoritmo genético com elitismo para otimizar cardápios alimentares de acordo com metas nutricionais."""

    populacao = copy.deepcopy(populacao_inicial)
    historico_fitness = []

    # Para cada geração...
    for g in range(num_geracoes):
        # Calcula o fitness de cada indivíduo da população
        fitnesses = [
            calcular_fitness(calcular_nutrientes_cardapio(c, df), metas)
            for c in populacao
        ]

        # Armazena o melhor fitness da geração atual no histórico
        historico_fitness.append(max(fitnesses))

        # Elitismo: encontra o melhor indivíduo da geração atual
        melhor_idx = fitnesses.index(max(fitnesses))
        elite = copy.deepcopy(populacao[melhor_idx])

        # Seleciona indivíduos para reprodução usando torneio
        selecionados = selecao_por_torneio(populacao, fitnesses)

        nova_populacao = []
        # Gera N-1 novos indivíduos (deixando espaço para o elite)
        for i in range(0, tamanho_populacao - 1, 2):
            pai1 = selecionados[i]
            # Garante que o índice não ultrapasse o tamanho da população
            pai2 = selecionados[i + 1 if i + 1 < tamanho_populacao else 0]
            # Realiza o crossover entre os pais para gerar dois filhos
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            # Aplica mutação nos filhos e adiciona à nova população
            nova_populacao.extend(
                [mutacao_refeicao(filho1, df), mutacao_refeicao(filho2, df)]
            )
        nova_populacao = nova_populacao[: tamanho_populacao - 1]

        # Adiciona o indivíduo elite à nova população
        nova_populacao.append(elite)

        # Atualiza a população para a próxima geração, garantindo o tamanho correto
        populacao = nova_populacao[:tamanho_populacao]

    # Após todas as gerações, calcula o fitness final de cada indivíduo
    fitnesses = [
        calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao
    ]

    # Encontra o índice do melhor indivíduo da população final
    melhor_idx = fitnesses.index(max(fitnesses))

    # Retorna o melhor cardápio, seu fitness e o histórico de fitness
    return populacao[melhor_idx], fitnesses[melhor_idx], historico_fitness

In [ ]:
melhor_cardapio, melhor_fitness, historico_fitness_elitismo = algoritmo_genetico_com_elitismo()

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_base,
    color="red",
    linestyle="-",
    linewidth=2,
    label="Sem Elitismo",
)
plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_elitismo,
    color="blue",
    linestyle="-",
    linewidth=2,
    label="Com Elitismo",
)
plt.legend()
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Melhor Fitness por Geração")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def algoritmo_genetico_com_elitismo_mutacao_por_refeicao(
    populacao_inicial=populacao_inicial,
    df=df_alimentos,
    metas=METAS_NUTRICIONAIS,
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES,
):
    """Executa um algoritmo genético com elitismo e mutação por refeição
    para otimizar cardápios alimentares de acordo com metas nutricionais."""

    populacao = copy.deepcopy(populacao_inicial)
    historico_fitness = []

    # Para cada geração...
    for g in range(num_geracoes):
        # Calcula o fitness de cada indivíduo da população
        fitnesses = [
            calcular_fitness(calcular_nutrientes_cardapio(c, df), metas)
            for c in populacao
        ]

        # Armazena o melhor fitness da geração atual no histórico
        historico_fitness.append(max(fitnesses))

        # Elitismo: identifica o melhor indivíduo da geração atual
        melhor_idx = fitnesses.index(max(fitnesses))
        elite = copy.deepcopy(populacao[melhor_idx])

        nova_populacao = []
        # Seleciona indivíduos para reprodução usando torneio
        selecionados = selecao_por_torneio(populacao, fitnesses)

        # Gera N-1 novos indivíduos (deixando espaço para o elite)
        for i in range(0, tamanho_populacao - 1, 2):
            pai1 = selecionados[i]
            # Garante que o índice não ultrapasse o tamanho da população
            pai2 = selecionados[i + 1 if i + 1 < tamanho_populacao else 0]
            # Realiza o crossover entre os pais para gerar dois filhos
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            # Aplica mutação por alimento em cada filho e adiciona à nova população
            nova_populacao.extend(
                [mutacao_alimento(filho1, df), mutacao_alimento(filho2, df)]
            )
        nova_populacao = nova_populacao[: tamanho_populacao - 1]

        # Adiciona o indivíduo elite à nova população
        nova_populacao.append(elite)

        # Atualiza a população para a próxima geração
        populacao = nova_populacao[:tamanho_populacao]

    # Após todas as gerações, calcula o fitness final de cada indivíduo
    fitnesses = [
        calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao
    ]

    # Identifica o índice do melhor indivíduo da população final
    melhor_idx = fitnesses.index(max(fitnesses))

    # Retorna o melhor cardápio, seu fitness e o histórico de fitness
    return populacao[melhor_idx], fitnesses[melhor_idx], historico_fitness

In [ ]:
melhor_cardapio, melhor_fitness, historico_fitness_elitismo_mutacao_por_refeicao = algoritmo_genetico_com_elitismo_mutacao_por_refeicao()

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")

In [ ]:
plt.figure(figsize=(20, 4))

plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_base,
    color="red",
    linestyle="-",
    linewidth=2,
    label="Sem Elitismo",
)
plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_elitismo,
    color="blue",
    linestyle="-",
    linewidth=2,
    label="Com Elitismo",
)
plt.plot(
    range(1, NUM_GERACOES + 1),
    historico_fitness_elitismo_mutacao_por_refeicao,
    color="green",
    linestyle="-",
    linewidth=2,
    label="Com Elitismo e Mutação por Alimento",
)
plt.legend()
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Melhor Fitness por Geração")
plt.grid(True, alpha=0.3)
plt.show()